In [57]:
import cv2
from ultralytics import YOLO
import cvzone
import math

from sort import *

In [58]:
# creating web cam object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Videos/cars.mp4')
#cap.set(3,1280) # for width
#cap.set(4,720) #for height

In [59]:
model = YOLO('./Yolo-Weights/yolov8n.pt')

In [60]:
# classes that will be detected and displayed
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush" , "pen"
              ]

In [61]:
mask = cv2.imread('./mask.png')

In [62]:
# creating a instance for tracking
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)

# we will make a line and count after that 
limits = [423,297,673,297]

In [63]:
while True:
    success, img = cap.read()
    #overlay mask on the image/webcam feed
    imageRegion = cv2.bitwise_and(img,mask)
    # results = model(img,stream=True)
    results = model(imageRegion,stream=True)
    
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # xy1 are points and xy2 are height and width
            x1,y1,x2,y2 = box.xyxy[0]
            # converting them into integers instead for tensor 
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3) #using cv2 reactangle
            w,h = x2-x1 , y2-y1
            bbox = int(x1),int(y1),int(w),int(h)
            
            # print(x1,y1,x2,y2)
            
            # cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
            #confidence
            conf = math.ceil((box.conf[0]*100))/100 #for confidence values
            print(conf)
            #classnames
            cls = int(box.cls[0])
            #to display conf and object class on webcam
            currentclass = classNames[cls]
            
            if currentclass == "car" or currentclass == "truck" or currentclass == "bus" or currentclass == "motorbike" and conf>0.3 :
                # cvzone.putTextRect(img,f'{currentclass} {conf}',(max(0,x1),max(35,y1)) , scale=0.6,thickness=1,offset=3)
                cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=5) #using cvzone rectange
                currentArrays = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArrays))
            
    resultsTracker = tracker.update(detections)
    
    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,255),5)
    
    for result in resultsTracker:
        x1,y1,x2,y2,id = result
        # converting them into integers instead for tensor 
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h = x2-x1 , y2-y1
        cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=2,colorR=(255,0,0))
        cvzone.putTextRect(img,f' {int(id)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
    
    cv2.imshow("Image",img)
    # cv2.imshow("ImageRegion",imageRegion)
    # cv2.waitKey(1)
    # Wait for a key event, but only for a short duration (10 milliseconds)
    # If the key is Esc (27), then break out of the loop and close the webcam
    if cv2.waitKey(0) == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

0.82
0.79
0.49
0.39


0: 384x640 4 cars, 146.2ms
Speed: 17.8ms preprocess, 146.2ms inference, 16.3ms postprocess per image at shape (1, 3, 384, 640)


[        572         322         651         429         245]
[        573         364         626         429         244]
[        378         348         497         457         243]
[        456         225         510         270         242]



0: 384x640 4 cars, 85.1ms
Speed: 3.3ms preprocess, 85.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


0.8
0.77
0.53
0.52
[     568.48      320.73      651.52      436.27         245]
[     572.26      371.69      627.74      438.31         244]
[     373.12      349.89      492.88      459.11         243]
[     454.74      228.22      508.26      273.78         242]


...

[     563.46      201.09      610.41      249.24         278]
[     299.43      326.52      415.83      428.23         277]
[     504.22      209.96      546.25       244.7         276]
[     593.03      326.21       662.1      393.99         275]



0: 384x640 5 cars, 75.3ms
Speed: 2.1ms preprocess, 75.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)



0.86
0.8
0.79
0.68
0.63
[     560.08       204.6      609.39      255.96         278]
[     276.08       338.6      401.71      448.71         277]
[     501.88      213.05      543.93      249.29         276]
[     580.74      348.67      653.42      422.36         275]


0: 384x640 5 cars, 87.5ms
Speed: 2.5ms preprocess, 87.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


0.72
0.72
0.68
0.67
0.67
[     631.58      250.22      670.47      304.12         279]
[     555.89      206.04      608.82      261.91         278]
[     249.68      353.25         383      465.83         277]
[     499.35      217.02      541.51      254.03         276]
[      568.4      375.99      641.49      454.21         275]
